<a href="https://colab.research.google.com/github/gkaahara/DIO-BairesDev-ML/blob/main/Desafio%20de%20Treinamento%20com%20rede%20YOLO/Desafio_Treinamento_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
yaml_content = """
train: /content/coco/images/train2017
val: /content/coco/images/val2017

nc: 2  # Número de classes personalizadas
names: ['bicycle', 'dog']  # Substitua pelos nomes das classes desejadas
"""

with open('coco_custom.yaml', 'w') as file:
    file.write(yaml_content)


In [3]:
# %% [download reduzido e corrigido]
import os
import random
from pycocotools.coco import COCO
from tqdm import tqdm
from IPython import get_ipython

# Criar diretórios
os.makedirs('/content/coco/images/train2017', exist_ok=True)
os.makedirs('/content/coco/images/val2017', exist_ok=True)
os.makedirs('/content/coco/annotations', exist_ok=True)

# Baixar APENAS anotações (são pequenas)
if not os.path.exists("/content/coco/annotations/instances_train2017.json"):
    if not os.path.exists("annotations_trainval2017.zip"):
        print("Baixando anotações...")
        !curl -L "http://images.cocodataset.org/annotations/annotations_trainval2017.zip" -o "annotations_trainval2017.zip"
    !unzip -q annotations_trainval2017.zip -d /content/coco/

# Carregar anotações para filtrar imagens relevantes
coco_train = COCO('/content/coco/annotations/instances_train2017.json')
coco_val = COCO('/content/coco/annotations/instances_val2017.json')

# Classes desejadas e seus IDs
class_names = ['bicycle', 'dog']
class_ids = coco_train.getCatIds(catNms=class_names)

# Função melhorada para filtrar e baixar imagens
def download_subset(coco, split='train2017', sample_ratio=0.1):
    # Obter IDs de imagens que contêm nossas classes
    img_ids = list(set([ann['image_id'] for ann in coco.loadAnns(coco.getAnnIds(catIds=class_ids))]))

    # Amostrar aleatoriamente
    sample_size = max(1, int(len(img_ids) * sample_ratio))  # Garante pelo menos 1 imagem
    sampled_ids = random.sample(img_ids, sample_size)

    # Baixar apenas imagens selecionadas com tratamento de erros
    for img_id in tqdm(sampled_ids, desc=f"Baixando {split}"):
        try:
            img = coco.loadImgs(img_id)[0]
            img_url = f"http://images.cocodataset.org/{split}/{img['file_name']}"
            img_path = f"/content/coco/images/{split}/{img['file_name']}"

            # Verificar se a imagem já foi baixada
            if not os.path.exists(img_path):
                !curl -sL "{img_url}" -o "{img_path}"

                # Verificar se o download foi bem-sucedido
                if not os.path.exists(img_path) or os.path.getsize(img_path) == 0:
                    print(f"\nErro no download: {img['file_name']}")
                    if os.path.exists(img_path):
                        os.remove(img_path)
        except Exception as e:
            print(f"\nErro processando imagem ID {img_id}: {str(e)}")

# Executar para train e val com tratamento de erros
print("Iniciando download do conjunto de treino...")
download_subset(coco_train, 'train2017')

print("\nIniciando download do conjunto de validação...")
download_subset(coco_val, 'val2017', 0.2)  # 20% para validação

print("\nDownload concluído! Verificando estrutura...")
!find /content/coco/images/ -type f | wc -l

Baixando anotações...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241M  100  241M    0     0  94.5M      0  0:00:02  0:00:02 --:--:-- 94.5M
loading annotations into memory...
Done (t=20.98s)
creating index...
index created!
loading annotations into memory...
Done (t=2.13s)
creating index...
index created!
Iniciando download do conjunto de treino...


Baixando train2017: 100%|██████████| 749/749 [02:18<00:00,  5.40it/s]



Iniciando download do conjunto de validação...


Baixando val2017: 100%|██████████| 63/63 [00:12<00:00,  4.96it/s]


Download concluído! Verificando estrutura...
812


In [4]:
# %% [criação de yaml otimizado]
import glob

def create_yaml():
    with open('coco_custom.yaml', 'w') as f:
        f.write("path: /content/coco\n")
        f.write("train: images/train2017\n")
        f.write("val: images/val2017\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write(f"names: {class_names}\n")

create_yaml()
create_yaml()

In [14]:
import os
import json
from tqdm import tqdm
from pycocotools.coco import COCO

# Caminhos
coco_annotations_path = "/content/coco/annotations/instances_val2017.json"  # Arquivo COCO JSON de validação
output_labels_dir = "/content/coco/labels/val2017/"  # Diretório para salvar os rótulos no formato YOLO

# Criação do diretório de saída para rótulos
os.makedirs(output_labels_dir, exist_ok=True)

# Carregar COCO
coco = COCO(coco_annotations_path)

# Classes desejadas
classes = ['bicycle', 'dog']
class_ids = []
for cls in classes:
    ids = coco.getCatIds(catNms=cls)
    if ids:
        class_ids.append(ids[0])
    else:
        print(f"Warning: Class '{cls}' not found in annotations.")

print(f"Converting annotations for classes with IDs: {class_ids}...")

# Gerar anotações YOLO
img_ids = coco.getImgIds()
for img_id in tqdm(img_ids):
    img = coco.loadImgs(img_id)[0]
    annotations = coco.loadAnns(coco.getAnnIds(imgIds=img_id, catIds=class_ids, iscrowd=False))

    # Ignorar images without the desired classes
    if not annotations:
        continue

    # Criar arquivo .txt para a imagem
    label_file = os.path.join(output_labels_dir, f"{os.path.splitext(img['file_name'])[0]}.txt")
    with open(label_file, 'w') as f:
        for ann in annotations:
            x, y, w, h = ann['bbox']
            cx, cy = x + w / 2, y + h / 2
            # Find the index of the category_id within the class_ids list
            try:
                label = class_ids.index(ann['category_id'])
                f.write(f"{label} {cx/img['width']} {cy/img['height']} {w/img['width']} {h/img['height']}\n")
            except ValueError:
                # This case should theoretically not happen if catIds is used in getAnnIds,
                # but as a safeguard
                print(f"Warning: Annotation with category_id {ann['category_id']} found but not in desired class_ids.")


print(f"Conversion complete. Labels saved in {output_labels_dir}")

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
Converting annotations for classes with IDs: [2, 18]...


100%|██████████| 5000/5000 [00:00<00:00, 60423.42it/s]

Conversion complete. Labels saved in /content/coco/labels/val2017/


In [6]:
ls /content/coco/labels/val2017/


000000000139.txt  000000147518.txt  000000289586.txt  000000435081.txt
000000000285.txt  000000147725.txt  000000289594.txt  000000435205.txt
000000000632.txt  000000147729.txt  000000289659.txt  000000435206.txt
000000000724.txt  000000147740.txt  000000289702.txt  000000435208.txt
000000000776.txt  000000147745.txt  000000289741.txt  000000435299.txt
000000000785.txt  000000148508.txt  000000289938.txt  000000435880.txt
000000000802.txt  000000148620.txt  000000289960.txt  000000436315.txt
000000000872.txt  000000148662.txt  000000289992.txt  000000436551.txt
000000000885.txt  000000148707.txt  000000290081.txt  000000436617.txt
000000001000.txt  000000148719.txt  000000290163.txt  000000436738.txt
000000001268.txt  000000148730.txt  000000290179.txt  000000436883.txt
000000001296.txt  000000148739.txt  000000290248.txt  000000437110.txt
000000001353.txt  000000148783.txt  000000290293.txt  000000437205.txt
000000001425.txt  000000148957.txt  000000290592.txt  000000437239.txt
000000

In [7]:
from pycocotools.coco import COCO
import shutil

# Caminho das anotações
annotation_file = '/content/coco/annotations/instances_train2017.json'
output_dir = '/content/coco/labels/train2017'

os.makedirs(output_dir, exist_ok=True)

# Carregar COCO e classes desejadas
coco = COCO(annotation_file)
classes = ['bicycle', 'dog']
class_ids = [coco.getCatIds(catNms=cls)[0] for cls in classes]

# Gerar anotações YOLO
for img_id in coco.getImgIds():
    img = coco.loadImgs(img_id)[0]
    annotations = coco.loadAnns(coco.getAnnIds(imgIds=img_id, catIds=class_ids, iscrowd=False))

    # Ignorar imagens sem as classes desejadas
    if not annotations:
        continue

    # Criar arquivo .txt para a imagem
    label_file = os.path.join(output_dir, f"{os.path.splitext(img['file_name'])[0]}.txt")
    with open(label_file, 'w') as f:
        for ann in annotations:
            x, y, w, h = ann['bbox']
            cx, cy = x + w / 2, y + h / 2
            label = class_ids.index(ann['category_id'])
            f.write(f"{label} {cx/img['width']} {cy/img['height']} {w/img['width']} {h/img['height']}\n")


loading annotations into memory...
Done (t=23.93s)
creating index...
index created!


In [9]:
!ls /content/coco/images/train2017 | wc -l  # Deve ser > 0
!ls /content/coco/labels/train2017 | wc -l  # Deve ser > 0

749
7497


In [10]:
!ls /content/coco/images/train2017 | wc -l  # Deve ser > 0
!ls /content/coco/labels/train2017 | wc -l  # Deve ser > 0

749
7497


In [15]:
# %% [treinamento rápido]
from ultralytics import YOLO

model = YOLO('yolov5s.pt')  # Modelo pré-treinado

# Hiperparâmetros otimizados para treino rápido
results = model.train(
    data='coco_custom.yaml', # This now has 'train' and 'val' keys
    epochs=10,               # Reduzido de 30
    imgsz=416,               # Menor que 640 para velocidade
    batch=8,                 # Tamanho reduzido
    workers=2,               # Menos workers para CPU
    patience=3,              # Early stopping
    cache='ram'              # Acelera carregamento
)

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco_custom.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.

train: Scanning /content/coco/labels/train2017.cache... 749 images, 0 backgrounds, 0 corrupt: 100%|██████████| 749/749 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.3GB RAM): 100%|██████████| 749/749 [00:04<00:00, 181.69it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1892.1±571.1 MB/s, size: 162.6 KB)


...

       4/10         0G      2.101      2.519      1.836          7        416: 100%|██████████| 94/94 [10:57<00:00,  7.00s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:16<00:00,  4.09s/it]

                   all         63        105      0.149     0.0842     0.0593     0.0254
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



4 epochs completed in 0.758 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 18.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 18.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv5s summary (fused): 84 layers, 9,112,310 parameters, 0 gradients, 23.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:16<00:00,  4.02s/it]


                   all         63        105      0.172      0.309       0.14     0.0555
               bicycle         28         62      0.206      0.129     0.0922     0.0301
                   dog         35         43      0.139      0.488      0.188      0.081
Speed: 1.2ms preprocess, 246.7ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train3
